In [55]:
from datasets import load_dataset, load_from_disk
import wordninja
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import re
import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from transformers import AutoTokenizer, default_data_collator, get_scheduler
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForPreTraining
from hf_transformers.src.transformers.models.bert.configuration_bert import BertConfig

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

import math

import collections


from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [ ]:
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForSequenceClassification

# Download configuration from huggingface.co and cache.
#Unilog元論文の実験に準拠
#https://arxiv.org/pdf/2112.03159.pdf
unilogConfig= BertConfig(
    is_unilog=True,
    attention_probs_dropout_prob=0.3,
    hidden_dropout_prob=0.3,
    num_attention_heads=4,
    hidden_size=128,
    intermediate_size=512,
    vocab_size=log_tokenizer_w_n.vocab_size,
    num_hidden_layers=3,
)
unilogConfigmodel = BertForSequenceClassification.from_pretrained("./logdata/unilog_pretrain_preln_on_attentions_0/", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./logdata/unilog_pretrain_preln_on_attentions_0/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [82]:
unilogConfigmodel

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4075, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.3, inplace=False)
            )
            (output): UnilogBertSelfOutput(
              (logact): LogACT(
                (linear

In [2]:
!export HF_DATASETS_CACHE="/home/"

In [3]:
#論文中の記述(4ページ), "tokenizer based on English Wikipedia unigram frequencies"
#Huggingfaceのトークナイザで上記に該当するの
    #bert-base-uncased**
    #roberta-base
    #distilbert-base-uncased
    #distilbert-base-multilingual-cased
#**実験に利用
#unigram freqencies?tokenizerのアルゴリズムは結局何なんだろう? Unigram or WordPiece?
#
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize_function(example):
    """
    A function that tokenizes all logs
    """
    return tokenizer(example["log"], truncation=True)


In [3]:
target = []
#NOTE
#ここで記号を区切ってwordninjaで塊除去を行っている
#なお, かなり時間がかかるので実行には注意が必要
#もう少し賢く書きたかったが, その書き方を思いつく時間とナイーブに実行する時間はどうせ同じぐらいだろう
def preprocess_log(text):
    text = text.replace('[', " ")
    text = text.replace(".", " ")
    text = text.replace(",", " ")
    text = text.replace(":", " ")
    text = text.replace("/", " ")
    text = text.replace(";", " ")
    text = text.replace("=", " ")
    text = text.replace("*", " ")
    text = text.replace("_", " ")
    text = text.replace("-", " ")
    text = text.lower()
    text = " ".join(wordninja.split(text))
    remove_num = lambda eg : " ".join([word for word in eg.split(" ") if not word.isdigit()])
    return remove_num(text)

In [19]:
print(hdfs[100])
print(preprocess_log(hdfs[100]))

2602297 node-175 unix.hw state_change.unavailable 1074300564 1 Component State Change: Component \042alt0\042 is in the unavailable state (HWID=3389)
node unix h w state change unavailable component state change component alt is in the unavailable state h wi d


In [190]:
# #訓練データとテストデータに分ける
# TRAIN_TEST_SPLIT_RATE = 0.8
# df_log["is_train"] = np.array([np.random.rand() < TRAIN_TEST_SPLIT_RATE for _ in range(len(df_log))])
# df_log[~df_log.is_train].to_csv("./logdata/test_logs.csv")
# df_log[df_log.is_train].to_csv("./logdata/train_logs.csv")

# データセットのロード

In [6]:
dataset = load_dataset("csv", data_files = "./logdata/all_logs.csv", cache_dir="/home/")

Found cached dataset csv (/home/csv/default-a5653168cf0fdeb7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00,  6.84it/s]


In [4]:
dataset = dataset["train"].train_test_split(test_size=0.2)

In [7]:
#データセットのトークナイズ
# tokenized_datasets = shuffled_train.map(tokenize_function, batched=True)

In [5]:
train_dataset = dataset["train"]

def get_training_corpus():
    for i in tqdm(range(0, len(train_dataset), 1000)):
        yield train_dataset[i : i + 1000]["log"]
        
def get_training_corpus_without_numbers():
    for i in tqdm(range(0, len(train_dataset), 1000)):
        yield train_dataset[i : i + 1000]["log_without_numbers"]
        
def get_training_corpus_small():
    for i in tqdm(range(0, len(train_dataset_small_samples), 1000)):
        yield train_dataset_small_samples[i : i + 1000]["log"]

# オリジナルのトークナイザからの訓練(上手くいきませんでした)

In [21]:
# pre_tokenizer = pre_tokenizers.WhitespaceSplit()
# pre_tokenizer.pre_tokenize_str(train_dataset[1000]["log"])

# tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
# special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]

# trainer = trainers.WordPieceTrainer(
#     vocab_size=5000, special_tokens=special_tokens, unk_token="<unk>"
# )
# tokenizer.train_from_iterator(get_training_corpus_small(), trainer=trainer)
# tokenizer.save(f"./log_tokenizer.json")

# encoding = tokenizer.encode(train_dataset[100]["log"])
# print(encoding.tokens)

# BERTトークナイザからの訓練

In [53]:
# old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# old_tokenizer(train_dataset[1000]["log"])
# new_tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus_without_numbers(), 25000)
# new_tokenizer.save_pretrained(f"./log_tokenizer_from_old_without_numbers_large")
# new_tokenizer(train_dataset[1000]["log"])

In [59]:
MODEL_MAX_LENGTH = 180
log_tokenizer = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_large/", model_max_length=MODEL_MAX_LENGTH)
log_tokenizer_w_n = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_without_numbers/", model_max_length=MODEL_MAX_LENGTH, padding=True)

# ログから数字除去したコード

In [ ]:
# def remove_numerical(example):
#     remove_num = lambda eg : " ".join([word for word in eg.split(" ") if not word.isdigit()])
#     example["log"] = [remove_num(eg) for eg in example["log"]]
#     return example

# dataset_without_numbers = dataset.map(remove_numerical, batched=True)
# train_log_without_numbers = train_dataset_without_numbers.to_pandas()
# train_log_without_numbers.set_index(train_log_without_numbers["Unnamed: 0"])
# train_log_without_numbers = train_log_without_numbers.iloc[:, 1:]
# ds_without_numbers = dataset["train"].add_column("log_without_numbers", dataset_without_numbers["train"]["log"])
# df_without_numbers = ds_without_numbers.to_pandas()
# df_without_numbers

In [28]:
lm_datasets_from_disk = load_from_disk("./logdata/logdata_for_pretraining/")

In [32]:
import copy

#Unilog元論文(https://arxiv.org/pdf/2112.03159.pdf)の表2の連続マスクを作成するための関数
#ランダムでマスクするインデックスの前後もマスクするようにした
def make_consecutive_mask(mask):
    new_mask = copy.deepcopy(mask)
    
    masked_indexes = np.where(mask)[0]
    
    for i, idx in enumerate(masked_indexes):
        if idx - 1 >= 0 and idx + 1 < len(new_mask):
            if i == 0 or abs(idx - masked_indexes[i-1]) > 3:
                new_mask[idx - 1] = 1
                new_mask[idx + 1] = 1
            else:
                new_mask[idx] = 0
        else:
            new_mask[idx] = 0
    return new_mask

In [53]:
#参考https://huggingface.co/learn/nlp-course/chapter7/3?fw=pt
def consecutive_word_masking_data_collator(features, wwm_probability=0.10):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        # CHANGED
        # 論文準拠にマスクを連続的にした
        mask = make_consecutive_mask(mask)
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = log_tokenizer_w_n.mask_token_id
        feature["labels"] = new_labels
    # print(features)
    return default_data_collator(features)

#バッチ単位でUnilogのワードマスキングを行う関数
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = consecutive_word_masking_data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [34]:
samples = [lm_datasets_from_disk[i] for i in range(2)]
batch = consecutive_word_masking_data_collator(samples, wwm_probability=0.05)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {log_tokenizer_w_n.decode(chunk)}'")


'>>> [CLS] info df s data node packet responder received [MASK] [MASK] [MASK] of size from [SEP] [CLS] r [MASK] [MASK] [MASK] j u r m n c j u ras kernel info microseconds spent in the rb s signal handler during calls microseconds [MASK] [MASK] [MASK] time for a single instance of a correctable ddr [SEP] [CLS] info df s fs data set deleting block b lk file m nt had oop df s data current sub dir b lk [SEP] [CLS] info df s fs name system block name system add stored block block map updated [MASK] [MASK] [MASK] b lk size [SEP] [CLS] info df s data node data x ce iver [MASK] [MASK] [MASK] lk s rc de st [SEP] [CLS] info df s fs data set deleting block b lk file m nt had oop df s data current sub dir b lk [SEP] [CLS] r m ne c j u r m ne c j u ras kernel info torus receiver z input pipe error s d cr [MASK] [MASK] [MASK] and corrected [SEP] [CLS]'

'>>> info df s fs name system block name system delete [MASK] [MASK] is added to invalid set of [SEP] [CLS] info df s data node packet responder re

# 訓練用のパラメータ設定

In [43]:
lm_datasets = lm_datasets_from_disk.train_test_split(test_size=0.1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128

In [48]:
train_dataset = load_from_disk("./logdata/logdata_for_pretraining_train/")
eval_dataset = load_from_disk("./logdata/logdata_for_pretraining_test/")

In [54]:
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=consecutive_word_masking_data_collator
)

eval_dataset = eval_dataset.map(
    insert_random_mask,
    batched=True,
    remove_columns=eval_dataset.column_names,
)

eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_token_type_ids": "token_type_ids",
        "masked_labels": "labels",
    }
)

eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

In [64]:
#Unilog元論文の実験に準拠
#https://arxiv.org/pdf/2112.03159.pdf
unilogConfig= BertConfig(
    is_unilog=True,
    attention_probs_dropout_prob=0.3,
    hidden_dropout_prob=0.3,
    num_attention_heads=4,
    hidden_size=128,
    intermediate_size=512,
    vocab_size=log_tokenizer_w_n.vocab_size,
    num_hidden_layers=3
)
unilogConfig

BertConfig {
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "is_only_mlm": false,
  "is_unilog": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4075
}

In [65]:
bertConfig= BertConfig(
    attention_probs_dropout_prob=0.3,
    hidden_dropout_prob=0.3,
    num_attention_heads=4,
    hidden_size=128,
    intermediate_size=512,
    vocab_size=log_tokenizer_w_n.vocab_size,
    num_hidden_layers=12,
    is_only_mlm=True
)
bertConfig

BertConfig {
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "is_only_mlm": true,
  "is_unilog": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4075
}

In [66]:
# del model
model = BertForPreTraining(config=bertConfig).to(device)
optimizer = AdamW(model.parameters(), lr=5e-4)

num_train_steps = 2**16 / (len(lm_datasets["train"]) // batch_size)

num_train_epochs = int(np.ceil(num_train_steps))
decayRate = 0.96
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

In [67]:
print(num_train_epochs)
output_dir = "./logdata/bert_log_pretrain_12_layers"
print(len(train_dataloader))

7
10687


# 事前学習開始

In [69]:
progress_bar = tqdm(range(num_train_epochs))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    pbar = tqdm(train_dataloader)
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        pbar.set_description(f">>> Loss {loss}")
        pbar.update(1)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(loss.repeat(batch_size))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    # try:
    perplexity = math.exp(torch.mean(losses))
    # except OverflowError:
    #     perplexity = float("inf")
    progress_bar.update(1)
    progress_bar.write(f">>> Epoch {epoch}: Perplexity: {perplexity}")
    # Save and upload
    model.save_pretrained(output_dir + f"_{epoch}")
    # if accelerator.is_main_process:
        # tokenizer.save_pretrained(output_dir)
        # repo.push_to_hub(
        #     commit_message=f"Training in progress epoch {epoch}", blocking=False
        # )

In [70]:
import gc
progress_bar = tqdm(range(num_train_epochs))
for epoch in tqdm(range(6,3,-1)):
    model6 = BertForPreTraining.from_pretrained(f"./logdata/bert_log_pretrain_{epoch}/").to(device)
    gc.collect()
    model6.eval()
    losses = []

    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model6(**batch)

        loss = outputs.loss
        losses.append(loss.repeat(batch_size))
        # pbar.update(1)

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    # try:
    perplexity = math.exp(torch.mean(losses))
    # except OverflowError:
    #     perplexity = float("inf")

    progress_bar.write(f">>> Epoch {epoch}: Perplexity: {perplexity}")



  0%|          | 0/7 [00:33<?, ?it/s]
                                     

                               
  0%|          | 0/3 [00:52<?, ?it/s]                                         

 33%|███▎      | 1/3 [00:52<01:44, 52.20s/it]25/10687 [01:25<34:05,  5.21it/s]

>>> Epoch 6: Perplexity: 313.2866145731586


                                             

                               
 33%|███▎      | 1/3 [01:44<01:44, 52.20s/it]                                 

 67%|██████▋   | 2/3 [01:44<00:52, 52.39s/it]25/10687 [02:18<34:05,  5.21it/s]

>>> Epoch 5: Perplexity: 313.2866145731586


                                             

                               
 67%|██████▋   | 2/3 [02:36<00:52, 52.39s/it]                                 

100%|██████████| 3/3 [02:36<00:00, 52.32s/it]25/10687 [03:10<34:05,  5.21it/s]

>>> Epoch 4: Perplexity: 313.2866145731586
